In [20]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
links_main = []
for i in range(2):
    r = requests.get('https://www.redkaraoke.com/karaoke/celine-dion/my-heart-will-go-on/11827/recordings/p' + str(i))
    soup = BeautifulSoup(r.content, 'html.parser')
    for link in soup.find_all('a', attrs = {'class':'result result-recording'}):
        links_main.append('https://www.redkaraoke.com' + link.get('href'))

In [3]:
# import urllib.request
# for i in range(len(links)):
#     url = links[i]
#     file_name = str(i) + '.mp3'
#     urllib.request.urlretrieve(url, file_name)
#     print(i)

In [4]:
link_sub = links_main[1]
r = requests.get(link_sub)
soup = BeautifulSoup(r.content, 'lxml')
soup

<!DOCTYPE html>
<!--[if lt IE 7]>      <html xmlns="https://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]--><!--[if IE 7]>         <html xmlns="https://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml" class="no-js lt-ie9 lt-ie8"> <![endif]--><!--[if IE 8]>         <html xmlns="https://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml" class="no-js lt-ie9"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js" xmlns="https://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml"> <!--<![endif]-->
<head>
<title>Gracellen sings My heart will go on in the style of Celine Dion on Red Karaoke</title>
<meta content="Recording of My heart will go on in the style of Celine Dion by Gracellen. Have fun singing and share it with your friends!" name="description"/>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<link href="https://d4rgvppl

In [30]:
soup.find_all('script', attrs = {'type' : 'text/javascript'})

[<script type="text/javascript">
 
 $('body').on('touchstart.dropdown', '.dropdown-menu', function (e) { e.stopPropagation();});
 
 var normalize = function( term ) {
 	var ret = "";
 	for ( var i = 0; i < term.length; i++ ) {
 		ret += accentMap[ term.charAt(i) ] || term.charAt(i);
 	}
 	return ret;
 };
 $(function() {
 			function highlightText(text, $node) {
 				var searchText = $.trim(text).toLowerCase(), currentNode = $node.get(0).firstChild, matchIndex, newTextNode, newSpanNode;
 				currentNode.data = currentNode.data.replace(/İ/g, 'I');
 				while ((matchIndex = currentNode.data.toLowerCase().indexOf(searchText)) >= 0) {
 					newTextNode = currentNode.splitText(matchIndex);
 					currentNode = newTextNode.splitText(searchText.length);
 					newSpanNode = document.createElement("span");
 					newSpanNode.className = "highlight";
 					currentNode.parentNode.insertBefore(newSpanNode, currentNode);
 					newSpanNode.appendChild(newTextNode);
 				}
 			}
 		$("#autocomplete1").a

In [28]:
pattern = re.compile('var .file=')
scripts = soup.find_all('script')
for script in scripts:
   if(pattern.match(str(script.string))):
       data = pattern.match(script.string)
       print(data)

In [27]:
soup

<!DOCTYPE html>
<!--[if lt IE 7]>      <html xmlns="https://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]--><!--[if IE 7]>         <html xmlns="https://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml" class="no-js lt-ie9 lt-ie8"> <![endif]--><!--[if IE 8]>         <html xmlns="https://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml" class="no-js lt-ie9"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js" xmlns="https://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml"> <!--<![endif]-->
<head>
<title>Gracellen sings My heart will go on in the style of Celine Dion on Red Karaoke</title>
<meta content="Recording of My heart will go on in the style of Celine Dion by Gracellen. Have fun singing and share it with your friends!" name="description"/>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<link href="https://d4rgvppl

In [52]:
a = soup.find_all('script')
data = str(a).split("userRecording.file=")[-1].rsplit(';', 1)


In [54]:
data = str(a).split("userRecording.file=")[-1].split(';', 1)

data.split

AttributeError: 'list' object has no attribute 'split'